# CSCE 670 :: Information Storage and Retrieval :: Spring 2020

# Visdom - a AI training virtualization tool

## Introduction

Visdom is a virtualization tool for pytorch which developed by FACEBOOK Artificial Intelligence. It can be used to create, organize, and share real-time, rich data. Support Torch and Numpy and pytorch. Visdom realize the visualization of remote data, which is very helpful for scientific experiments. We can send pictures and data remotely and display them on the UI interface to check the experimental results or debug.

Here are some views show how visdom looks like when you train a machine learning and monitor the training using visdom.

<img src="visdom_chart.jpg" style="width: 500px;">
<img src="visdom_main.jpg" style="width: 500px;">

## Install
Python3 is required

`pip install visdom`

or

`conda install -c conda-forge visdom` if using Anaconda

### Test if visdom is successfully installed

Run `visdom` or `python -m visdom.server` on command-line.
If it's successfully installed, the screen should look like this.
<img src="run_visdom.png" style="width: 500px;">

Now you can visit the url presented on the terminal, but it should be blank at first.

## Concepts
### Panes: 
The interface is blank when first opening it. Transmitting plots and charts to backend can make it generate panes. Panes can be dragged, resized or deleted. Take the picture above for example, there are 12 generated panes, each with different chart type and style.

<img src="visdom_blank.png" style="width: 500px;">

### Environment: 
envs can divid a visualization space into groups. The visual results of different environments are isolated from each other and do not affect each other. If you do not specify env when using it, the default is to use main. Different users and different programs generally use different env.

<img src="envs.png" style="height: 100px;">

#### Select environments
You can select a environment from the bar on the top of the visdom page.

#### Erase Environments
On the bar of the visdom page, there's a erase button. Click on the erase button can clean out the whole content of the current environment. For preventing accidently clicks, it would ask you to confirm again after pressing the button.

#### Save Environments
Click the save button(near the erase button) to save the current env as a json file. The storage path is located in the ~ / .visdom / directory. You can also modify the name of the env and click fork to save the current env status to the renamed env.

### State: 

When you put graphs on the server, the server would cache those graph. So, if a user turn off the server and turn on it again, the graphs would still be there.

There are two points to note when using Visdom:
1. You need to manually specify the save env. For example, `vis.text('Hello, world!', env="main")`
you can click the save button on the web interface or call the save method in the program, otherwise the env and other information will be lost after the visdom service restarts
2. The interaction between the client and the server uses the tornado asynchronous framework, and the visual operation will not block the current program, and network abnormalities will not cause the program to exit.


### Drawing Functions
Visdom has several common drawing functions, such as:

-line: Similar to the 'plot' operation in Matlab, used to record changes of certain scalars, such as loss, accuracy, etc.

-image: Visualized image, which can be the input image, the image generated by GAN, or the information of the convolution kernel

-text: Used to record text information such as logs, supports html format

-scatter: Draw a scatter plotbar: draw a bar chart

-bar: Draw a bar chart

-pie: Draw a pie chart

## Usage

Start the server from command line

`visdom`

or

`visdom -m visdom.server`

### Command-Line Options
The following options can be provided to the server:

`-port`: The port to run the server on.

`-hostname` : The hostname to run the server on.

`-base_url` : The base server url (default = /).

`-env_path` : The path to the serialized session to reload.

`-logging_level` : Logging level (default = INFO). Accepts both standard text and numeric logging values.

`-readonly` : Flag to start server in readonly mode.

`-enable_login` : Flag to setup authentication for the sever, requiring a username and password to login.

`-force_new_cookie` : Flag to reset the secure cookie used by the server, invalidating current login cookies.
Requires -enable_login.

## Some Examples

In [1]:
#need to start visdom server in the commandline first

import visdom
import numpy as np
import math
vis = visdom.Visdom(env="main")


Setting up a new session...


In [2]:
#vis.text
vis.text('Hello, world!', opts=dict(title="text", env="main"))

#vis.image
vis.image(np.random.rand(3, 256, 256), opts=dict(title="random_image"))

#vis.pie
vis.pie(np.asarray([35, 25, 40]), opts=dict(title="pie", legend=["Front-End Engineer", "Back-End Engineer", "Machine Learning Engineer"]))

#vis.bar
vis.bar(np.random.rand(10), opts=dict(title="bar"))

#vis.line
vis.line(np.random.rand(10), opts=dict(title="line"))

#vis.stem
Y = np.linspace(0, 2 * math.pi, 70)
X = np.column_stack((np.sin(Y), np.cos(Y)))
vis.stem(
    X=X,
    Y=Y,
    opts=dict(title="stem", legend=['Sine', 'Cosine'])
)

'window_385cc637d7ba2c'

Now you can check the page on visdom, there should be several panes on it

## PyTorch Example

In this example, we use visdom to monitor the training loss, testing loss and accuracy of a CNN, the CNN is in charge of identifying the digital numbers in MNIST dataset. The visdom figures update in each epoch. Notice that here we use `update='append'` in `vis.line()`, otherwise it would override the previous results.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from visdom import Visdom

batch_size = 200
learning_rate = 0.01
epochs = 10

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                   transforms.ToTensor(),
                   # transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        # transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)


class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)
        return x


device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

viz = Visdom(env="MNIST")
viz.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
viz.line([[0.0, 0.0]], [0.], win='test', opts=dict(title='test loss&acc.', legend=['loss', 'acc.']))
global_step = 0

for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()

        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        global_step += 1
        viz.line([loss.item()], [global_step], win='train_loss', update='append')

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.argmax(dim=1)
        correct += pred.eq(target).float().sum().item()

    viz.line([[test_loss, correct / len(test_loader.dataset)]],
             [global_step], win='test', update='append')
    viz.images(data.view(-1, 1, 28, 28), win='x')
    viz.text(str(pred.detach().cpu().numpy()), win='pred',
             opts=dict(title='pred'))

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Setting up a new session...


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.299417
Train Epoch: 0 [20000/60000 (33%)]	Loss: 2.285022
Train Epoch: 0 [40000/60000 (67%)]	Loss: 2.259548

Test set: Average loss: 0.0111, Accuracy: 4480.0/10000 (45%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.228044
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.160003
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.076617

Test set: Average loss: 0.0097, Accuracy: 4833.0/10000 (48%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.984172
Train Epoch: 2 [20000/60000 (33%)]	Loss: 1.816321
Train Epoch: 2 [40000/60000 (67%)]	Loss: 1.700139

Test set: Average loss: 0.0074, Accuracy: 6043.0/10000 (60%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.519780
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.374087
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.235020

Test set: Average loss: 0.0060, Accuracy: 6418.0/10000 (64%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.103241
Train Epoch: 4 [20000/60000 (33%)]	Loss: 1.180342
Train Epoch: 4 [40000/60000 (67%)]	Loss: 1.058966

Test set: Average loss

<img src="NN_result.png" style="width: 500px;">

## Reference

1. https://github.com/facebookresearch/visdom
2. https://ai.facebook.com/tools/visdom/
3. https://blog.csdn.net/SHU15121856/article/details/88818539
4. https://zhuanlan.zhihu.com/p/34692106
5. https://github.com/noagarcia/visdom-tutorial
6. https://udacity.github.io/udacidrone/docs/tutorial-visdom.html